# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

In [29]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from math import log

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [103]:
def clean_sentence(sentence):
    
    # Cleaning unwanted characters on the sentence
    string = sentence.replace("@", " ")
    string = string.replace(" por ", " ")
    string = string.replace(" do ", " ")
    string = string.replace(" da ", " ")
    string = string.replace(" de ", " ")
    string = string.replace(" à ", " ")
    string = string.replace(" aos ", " ")
    string = string.replace(" o ", " ")
    string = string.replace("/", " ")
    string = string.replace("2018", " ")
    string = string.replace(" a ", " ")
    string = string.replace(" e ", " ")
    string = string.replace(" que ", " ")
    string = string.replace(" se ", " ")
    string = string.replace(":", " ")
    string = string.replace(";", " ")
    string = string.replace(",", " ")
    string = string.replace("?", " ")
    string = string.replace("(", " ")
    string = string.replace(")", " ")
    string = string.replace("\n", " ")
    string = string.replace("'", " ")
    string = string.replace('"', " ")
    string = string.replace("!", " ")
    string = string.lower()

    # Converting the sentence into a list of words
    string_list = string.split()
    
    # Returning the clean list
    return string_list

In [104]:
# Extracting the DataSet
df_train = pd.read_excel('smartfit_naivebayes.xlsx')

In [105]:
# Counting the total of each class
count_0 = df_train.Classifição[df_train.Classifição == 0].count()
count_1 = df_train.Classifição[df_train.Classifição == 1].count()
count_2 = df_train.Classifição[df_train.Classifição == 2].count()

total_count = df_train.Classifição.count()

# Discovering the probability of each class
P_0 = count_0/total_count
P_1 = count_1/total_count 
P_2 = count_2/total_count

print("Probabilidade de Negativo: {:.2f}%".format(P_0*100))
print("Probabilidade de Irrelevante: {:.2f}%".format(P_1*100))
print("Probabilidade de Positivo: {:.2f}%".format(P_2*100))

Probabilidade de Negativo: 16.67%
Probabilidade de Irrelevante: 50.67%
Probabilidade de Positivo: 32.67%


In [106]:
# Creating a list to find out our d factor of the LaPlace Smoothing
possible_words = []

for sentence in df_train.Treinamento:
    for word in clean_sentence(sentence):
        if word not in possible_words:
            possible_words.append(word)
            
d = len(possible_words)

In [107]:
# Creating a Dictionary with the words that appear in each Class of sentences, and how many times it appears
# Also Creates a variable with the total of words for that Class of Sentences

# Class 0
words_0 = {}
number_words_0 = 0

for sentence in df_train.Treinamento[df_train.Classifição == 0]:
    for word in clean_sentence(sentence):
        if word in words_0:
            words_0[word] += 1
        else:
            number_words_0 += 1
            words_0[word] = 1
            
# Class 1
words_1 = {}
number_words_1 = 0

for sentence in df_train.Treinamento[df_train.Classifição == 1]:
    for word in clean_sentence(sentence):
        if word in words_1:
            words_1[word] += 1
        else:
            number_words_1 += 1
            words_1[word] = 1
            
# Class 2
words_2 = {}
number_words_2 = 0

for sentence in df_train.Treinamento[df_train.Classifição == 2]:
    for word in clean_sentence(sentence):
        if word in words_2:
            words_2[word] += 1
        else:
            number_words_2 += 1
            words_2[word] = 1

In [108]:
def calc_prob(sentence, words, number_words, P, d, alpha=1):
    prob = P
    
    # Alphha factor for the LaPlace smoothing
    total = number_words + alpha*d
    for word in clean_sentence(sentence):
        occurancy = alpha
        if word in words:
            occurancy += words[word]
        prob *= (occurancy/total)
            
    return prob

In [109]:
def comparing_probs(prob_0, prob_1, prob_2):

    if prob_0 > prob_1 and prob_0 > prob_2:
        return 0
    elif prob_1 > prob_0 and prob_1 > prob_2:
        return 1
    else:
        return 2
    

In [110]:
# Testing the Classifier with the test set
df_test = pd.read_excel("smartfit_naivebayes.xlsx", sheet_name=1)

alpha = 1

classification = []
for sentence in df_test.Teste:
    prob_0 = calc_prob(sentence, words_0, number_words_0, P_0, d, alpha=alpha)
    prob_1 = calc_prob(sentence, words_1, number_words_1, P_1, d, alpha=alpha)
    prob_2 = calc_prob(sentence, words_2, number_words_2, P_2, d, alpha=alpha)
    classif = comparing_probs(prob_0, prob_1, prob_2)
    classification.append(classif)
    
df_test["Prediction"] = classification

In [111]:
count = 0
for i in range(len(df_test)):
    if df_test.loc[i].Classifição == df_test.loc[i].Prediction:
        count += 1

print("Porcentagem de Acerto: {:.2f}%".format(100*count/len(df_test)))

Porcentagem de Acerto: 56.50%


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
